In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
import seaborn as sns
sns.set_theme(style="darkgrid")

In [2]:
print(tf.__version__)

2.12.0


# Réseaux de neurones convolutifs pour la classification de documents

 > ℹ️ Inspiré de :
 > - https://keras.io/examples/nlp/pretrained_word_embeddings/
 > - https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/

<div class="alert alert-block alert-info">

🥅 **Objectifs**

- Savoir utiliser `keras` pour faire de l'apprentissage supervisé à partir de documents avec des réseaux de neurones convolutifs
- Savoir prétraiter les données pour le réseau
- Utiliser des plongement pré-entraînés
- Réaliser une validation croisée pour l'apprentissage
- Interpréter les résultats obtenus

🚨 **Consignes**

Les réponses aux questions doivent être données sur Moodle (Questionnaires "Réponses aux questions du TP"). Cela vous permettra d'obtenir un retour immédiat pour les questions fermées.

</div>

## 1. Chargement des données

In [3]:
!mkdir data
!wget -P data https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/winemag-fr_train.csv

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [3]:
# Lecture du fichier CSV
wine_df = pd.read_csv("data/winemag-fr_train.csv", sep=",", dtype={'description': 'object',
                                           'price': 'float64',
                                           'province': 'category',
                                           'variety': 'object'})

In [4]:
# Liste des classes
class_names = sorted(wine_df.province.unique().categories.to_list())
print("Classes :", class_names)
print("Nombre d'exemplaires :", len(wine_df))

Classes : ['Alsace', 'Beaujolais', 'Bordeaux', 'Burgundy', 'Champagne', 'France_Other', 'Languedoc-Roussillon', 'Loire_Valley', 'Provence', 'Rhône_Valley', 'Southwest_France']
Nombre d'exemplaires : 8132


In [5]:
# On associe à chaque classe un identifiant unique
class_index = {class_names[i]:i for i in range(len(class_names))}
class_index

{'Alsace': 0,
 'Beaujolais': 1,
 'Bordeaux': 2,
 'Burgundy': 3,
 'Champagne': 4,
 'France_Other': 5,
 'Languedoc-Roussillon': 6,
 'Loire_Valley': 7,
 'Provence': 8,
 'Rhône_Valley': 9,
 'Southwest_France': 10}

In [6]:
# On utilise uniquement la variété et les descriptions comme données d'entrée
X_train_variety = wine_df.variety.str.split('_')
X_train = X_train_variety.str.join(' ') + ' ' + wine_df.description
# Les noms des classes sont remplacées par leur identifiant (un entier positif)
y_train = wine_df.province.map(class_index)

In [8]:
X_train.head()

0    Chardonnay Ripe plush pineapple laces through ...
1    Rhône-style Red Blend If you're going to enjoy...
2    Chardonnay A concentrated, hugely rich wine wi...
3    Bordeaux-style Red Blend Chosen from a selecti...
4    Bordeaux-style Red Blend Very much in the food...
dtype: object

In [9]:
y_train.head()

0    3
1    6
2    3
3    2
4    2
Name: province, dtype: category
Categories (11, int64): [0, 1, 2, 3, ..., 7, 8, 9, 10]

## 2. Indexation du vocabulaire

Le vocabulaire sera constitué par les 8 000 mots les plus fréquents et les documents seront tronqués ou complétés de manière à faire 50 tokens de long. La vectorisation est réalisée à l'aide de [TextVectorization](https://keras.io/api/layers/preprocessing_layers/text/text_vectorization/) de la bibliothèque [Keras](https://keras.io/). Keras permet de construire, entraîner et évaluer différents types de réseaux de neurones.

In [10]:
def get_vectorizer(documents, max_voc_size=8000, max_seq_length=50, batch_size=128):
  vectorizer = TextVectorization(max_tokens=max_voc_size, 
                                 output_sequence_length=max_seq_length)
  # Création du jeu de données à partir de X_train et constitution de lots de 128 instances
  text_ds = tf.data.Dataset.from_tensor_slices(documents).batch(batch_size)
  # Création du vocabulaire à partir des données d'entrée
  vectorizer.adapt(text_ds)
  return vectorizer

In [11]:
keras_vectorizer = get_vectorizer(X_train)

Vocabulaire obtenu :

In [12]:
voc = keras_vectorizer.get_vocabulary()
print(len(voc))

8000


Affichage des 5 items les plus fréquents dans le vocabulaire :

In [13]:
voc[:5]

['', '[UNK]', 'and', 'the', 'a']

L'item 0 est réservé pour compléter les séquences (chaîne vide). L'item 1 est réservé aux mots hors vocabulaire (UNK).

On associe ensuite à chaque item du vocabulaire un identifiant unique :

In [14]:
word_index = dict(zip(voc, range(len(voc))))

Exemple de vectorisation d'une phrase

In [15]:
print("Texte initial", X_train.iloc[1])
output = keras_vectorizer([X_train.iloc[1]])
print("Vocabulaire dans le texte (15 premiers items) :")
for v in output.numpy()[0, :15]:
  print(v, keras_vectorizer.get_vocabulary()[v])



Texte initial Rhône-style Red Blend If you're going to enjoy this wine now, let it open up for a couple of hours to allow some of the nuanced notes to express themselves. Sultry aromas of boysenberry and blueberry are framed with complex additions of purple florals, sweet cured meat, vanilla bean and toasty oak. The palate is lush and mouthfilling, with fine but aggressive tannins and a beautiful minerality to the dark fruit core. Cocoa-dusted truffle flavors flood the long finish; hold until 2016–2020.
Vocabulaire dans le texte (15 premiers items) :
101 rhônestyle
15 red
12 blend
364 if
2858 youre
442 going
11 to
784 enjoy
9 this
7 wine
47 now
1279 let
10 it
186 open
140 up


## 3. Chargement de plongements de mots pré-entraînés

Nous allons utiliser des plongement de mots pré-entraînés pour représenter les mots du vocabulaire. 

Nous allons commencer par tester des plongements obtenus à partir de Wikipedia en anglais, d'une dimension de 300 et construits avec l'algorithme "Continuous Skipgram" de Gensim (cf. https://git.unistra.fr/dbernhard/ftaa_data/-/blob/main/README.md)

🚨 **Les plongements utilisés dépendent de la langue des textes. Ces plongements ne sont donc pas adaptés pour des textes dans une langue autre que l'anglais.** 🚨 

Téléchargement des données :

In [16]:
!wget -P data https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/model_6.txt

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Nous allons construire un dictionnaire qui associera chaque mot à sa représentation vectorielle :

In [19]:
def load_embeddings(embeddings_file):
  embeddings_index = {}
  with open(embeddings_file, 'r', encoding='utf8') as f:
      for line in f:
          word, coefs = line.split(maxsplit=1)
          coefs = np.fromstring(coefs, "f", sep=" ")
          embeddings_index[word] = coefs
  print(f'{len(embeddings_index)} vecteurs de mots ont été lus')
  return embeddings_index

In [21]:
# Chargement des plongements du fichier model_6.txt
m6_embeddings = load_embeddings('model_6.txt')

6737 vecteurs de mots ont été lus


Pour information, voici à quoi ressemble le fichier contenant les plongements pré-entraînés :

In [23]:
!head model_6.txt

'head' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Nous allons maintenant préparer une matrice de plongements. Dans cette matrice, la ligne `i` correspondra au plongement pré-entraîné pour le mot d'indice `i` dans le vocabulaire :

In [24]:
def get_embedding_matrix(vocabulary, embeddings_index, embedding_dim = 300):
  num_tokens = len(vocabulary)
  hits = 0
  misses = 0

  # Préparation de la matrice
  # Les mots qui ne se trouvent pas dans les plongements pré-entraînés seront 
  # représentés par des vecteurs dont toutes les composantes sont égales à 0,
  # y compris la représentation utilisée pour compléter les documents courts et
  # celle utilisée pour les mots inconnus [UNK]
  embedding_matrix = np.zeros((num_tokens, embedding_dim))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
          hits += 1
      else:
          misses += 1
  print(f'{hits} mots ont été trouvés dans les plongements pré-entraînés')
  print(f'{misses} sont absents')
  return embedding_matrix

In [25]:
# Construction de la matrice de plongements à partir du vocabulaire
m6_embedding_matrix = get_embedding_matrix(voc, m6_embeddings)

5281 mots ont été trouvés dans les plongements pré-entraînés
2719 sont absents


In [26]:
m6_embedding_matrix.shape

(8000, 300)

## 4. Construction et entraînement du modèle


Nous allons faire une validation croisée à 5 plis.

L'architecture du modèle est la suivante :    
- Entrée : documents représentés par la concaténation des plongements représentant les mots
  - Les données sont traitées par lots de 128 documents (`batch_size`) : la descente du gradient se fera sur un lot.
  - La représentation de chaque document a une longueur de 50 tokens (avec remplissage si nécessaire)
  - Chaque token est représenté par un plongement de dimension 300.
- Couche 1 : convolution avec 64 filtres et une fenêtre de 5 mots
  - Pour chacun des 64 filtres, on obtient une carte de 46 caractéristiques (dans 50 tokens, il y a 46 fenêtres de 5 tokens)
  - Nombre de paramètres à apprendre (300 * 5 + 1 ) * 64 = 96064
- Couche 2 : pooling maximum avec une fenêtre de 5 et un pas de 5 :    
  - Pour chaque carte de caractéristiques, on obtient un vecteur de dimension 9 (il y a 9 fenêtres de taille 5 qui ne se superposent pas dans un vecteur de 46 caractéristiques)
- Couche 3 : convolution avec 64 filtres et une fenêtre de 5
  - Pour chaque filtre, on obtient une carte de 5 caractéristiques (5 fenêtres de 5 caractéristiques dans un vecteur de 9 caractéristiques)
  - Nombre de paramètres à apprendre : (64 * 5 + 1 ) * 64 = 20544
- Couche 4 : pooling maximum global. 
  - On ne conserve que le maximum de chaque carte de caractéristiques
- Couche 5 : couche entièrement connectée avec 64 unités
  - Nombre de paramètres à apprendre : 64 * (64 + 1) = 4160
- Couche 6 : _Dropout_ 
  - Pour éviter le surajustement, 50% des neurones sont aléatoirement ignorés. De sorte, les neurones sont "forcés" à (1) avoir chacun leur propre utilité, car ils ne peuvent pas s'adapter avec leurs neurones voisins, et (2) prendre en compte tous les neurones d'entrée, et non pas se focaliser sur quelques uns.
- Couche 7 : couche entièrement connectée avec 11 unités (le nombre de classes)
  - Retourne une liste de 11 probabilités (une pour chacune des classes cibles)
  - Nombre de paramètres à apprendre : 11 * (64 + 1) = 715

In [27]:
def get_CNN_model(voc_size, embedding_matrix, embedding_dim=300):
  # Création du modèle
  int_sequences_input = keras.Input(shape=(None,), dtype="int64")
  embedding_layer = Embedding(voc_size, embedding_dim, trainable=True,
      embeddings_initializer=keras.initializers.Constant(embedding_matrix),
  )
  
  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Conv1D(64, 5, activation="relu")(embedded_sequences)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(64, 5, activation="relu")(x)
  x = layers.GlobalMaxPooling1D()(x)
  x = layers.Dense(64, activation="relu")(x)
  x = layers.Dropout(0.5)(x)
  preds = layers.Dense(len(class_names), activation="softmax")(x)
  model = keras.Model(int_sequences_input, preds)
  return model

In [28]:
# Affichage de l'architecture du modèle
m6_model = get_CNN_model(len(voc), m6_embedding_matrix)
m6_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         2400000   
                                                                 
 conv1d (Conv1D)             (None, None, 64)          96064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 64)          20544     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                              

In [29]:
# Fonction pour l'entraînement d'un modèle
def train_model(X, y, model_function, vectorizer,
                voc_size, embedding_matrix, embedding_dim=300, batch_size=128):
  # Listes utilisées pour sauvegarder les résultats obtenus à chaque pli
  acc_per_fold = []
  loss_per_fold = []
  histories = []
  folds = 5
  stratkfold = model_selection.StratifiedKFold(n_splits=folds, shuffle=True, 
                                              random_state=12)
  fold_no = 1
  for train, test in stratkfold.split(X, y):
    m_function = globals()[model_function]
    model = m_function(voc_size, embedding_matrix, embedding_dim)

    print('------------------------------------------------------------------------')
    print(f'Entraînement pour le pli {fold_no} ...')
    fold_x_train = vectorizer(X.iloc[train].to_numpy()).numpy()
    fold_x_val = vectorizer(X.iloc[test].to_numpy()).numpy()
    fold_y_train = y.iloc[train].to_numpy()
    fold_y_val = y.iloc[test].to_numpy()

    # Compilation du modèle : permet de préciser la fonction de perte et l'optimiseur
    # loss=sparse_categorical_crossentropy : entropie croisée, dans le cas où les 
    #  classes cibles sont indiquées sous forme d'entiers. Il s'agira de minimiser
    #  la perte pendant l'apprentissage
    # optimizer=rmsprop : l'optimiseur détermine la manière doit les poids seront
    #  mis à jour pendant l'apprentissage
    model.compile(
      loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
    )
    # Entraînement sur 10 époque (la totalité du jeu de données est parcourue
    # 10 fois)
    history = model.fit(fold_x_train, fold_y_train, batch_size=batch_size, 
                        epochs=10, validation_data=(fold_x_val, fold_y_val))
    histories.append(history)
    # Evaluation sur les données de validation
    scores = model.evaluate(fold_x_val, fold_y_val, verbose=0)
    print(f'Scores pour le pli {fold_no}: {model.metrics_names[0]} = {scores[0]:.2f};',
          f'{model.metrics_names[1]} = {scores[1]*100:.2f}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    fold_no = fold_no + 1

  # Affichage des scores moyens par pli
  print('---------------------------------------------------------------------')
  print('Scores par pli')
  for i in range(0, len(acc_per_fold)):
    print('---------------------------------------------------------------------')
    print(f'> Pli {i+1} - Loss: {loss_per_fold[i]:.2f}',
          f'- Accuracy: {acc_per_fold[i]:.2f}%')
  print('---------------------------------------------------------------------')
  print('Scores moyens pour tous les plis :')
  print(f'> Accuracy: {np.mean(acc_per_fold):.2f}',
        f'(+- {np.std(acc_per_fold):.2f})')
  print(f'> Loss: {np.mean(loss_per_fold):.2f}')
  print('---------------------------------------------------------------------')
  return histories

In [30]:
# Entraînement du modèle et récupération des résultats
CNN_histories = train_model(X_train, y_train, 'get_CNN_model',
                            keras_vectorizer, len(voc), m6_embedding_matrix)

------------------------------------------------------------------------
Entraînement pour le pli 1 ...
Epoch 1/10
51/51 [==============================] - 3s 43ms/step - loss: 2.1999 - acc: 0.2589 - val_loss: 1.9305 - val_acc: 0.4216
Epoch 2/10
51/51 [==============================] - 2s 40ms/step - loss: 1.6426 - acc: 0.4732 - val_loss: 1.3412 - val_acc: 0.5519
Epoch 3/10
51/51 [==============================] - 2s 41ms/step - loss: 1.2715 - acc: 0.6008 - val_loss: 1.0633 - val_acc: 0.6632
Epoch 4/10
51/51 [==============================] - 2s 43ms/step - loss: 1.0585 - acc: 0.6692 - val_loss: 0.9040 - val_acc: 0.7154
Epoch 5/10
51/51 [==============================] - 2s 41ms/step - loss: 0.8770 - acc: 0.7368 - val_loss: 0.7052 - val_acc: 0.7947
Epoch 6/10
51/51 [==============================] - 2s 38ms/step - loss: 0.7445 - acc: 0.7745 - val_loss: 0.6375 - val_acc: 0.8095
Epoch 7/10
51/51 [==============================] - 2s 38ms/step - loss: 0.6139 - acc: 0.8146 - val_loss: 0.58

Affichage des résultats sous forme graphique :

In [ ]:
def plot_results(histories):
  accuracy_data = []
  loss_data = []
  for i, h in enumerate(histories):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']
    for j in range(len(acc)):
      accuracy_data.append([i+1, j+1, acc[j], 'Entraînement'])
      accuracy_data.append([i+1, j+1, val_acc[j], 'Validation'])
      loss_data.append([i+1, j+1, loss[j], 'Entraînement'])
      loss_data.append([i+1, j+1, val_loss[j], 'Validation'])

  acc_df = pd.DataFrame(accuracy_data, 
                        columns=['Pli', 'Epoch', 'Accuracy', 'Données'])
  sns.relplot(data=acc_df, x='Epoch', y='Accuracy', hue='Pli', style='Données',
              kind='line')
    
  loss_df = pd.DataFrame(loss_data, columns=['Pli', 'Epoch', 'Perte', 'Données'])
  sns.relplot(data=loss_df, x='Epoch', y='Perte', hue='Pli', style='Données',
              kind='line')

In [ ]:
plot_results(CNN_histories)

La perte observée sur les données d'entraînement diminue à chaque époque, tandis que la justesse augmente. On n'observe toutefois pas la même tendance pour les données de validation : la perte et la justesse semblent stagner au bout de la 6<sup>ème</sup> époque. Ainsi, un modèle qui obtient de meilleures performances sur les données d'entraînement n'obtient pas forcément de bons résultats sur des données qui n'ont pas été utilisées pour l'apprentissage. Cela montre que le modèle est _surajusté_ aux données d'entraînement : on devrait donc stopper l'apprentissage au bout de 6 époques.

❓ [1] Que constatez-vous par rapport aux résultats obtenus précédemment pour ce jeu de données (tf-idf) ?

## 5. Comparaison de différents plongements

Nous avons pour l'heure utilisé un seul type de plongements (`model_6.txt`). Vous trouverez deux autres fichiers de plongements pré-entrainés dans le [dépôt de données](https://git.unistra.fr/dbernhard/ftaa_data) :      
- `model_26.txt`
- `glove_100.txt` (attention, ces derniers ont une dimension de 100 et non de 300, il faudra donc veiller à utiliser les bons paramètres pour les fonctions)

Refaites l'expérience en utilisant chacun de ces deux modèles. Pensez à bien sauvegarder les résultats précédents et veillez à réutiliser les fonctions existantes :
- `load_embeddings()` pour charger les plongements
- `get_embedding_matrix()` pour construire la matrice de plongements
- `train_model()` pour entraîner le modèle
- `plot_results()` pour afficher les résultats

❓ [2] Que constatez-vous ? Les plongements utilisés ont-ils une influence sur les résultats ?